In [10]:
import os
from dotenv import load_dotenv
import pandas as pd
from sqlalchemy import create_engine
from IPython.display import display
import bcrypt



In [11]:
load_dotenv()

DB_HOST = os.getenv("DB_HOST")
DB_PORT = os.getenv("DB_PORT")
DB_ENTRADA = os.getenv("DB_ENTRADA")
DB_SAI = os.getenv("DB_SAI")
DB_USER = os.getenv("DB_USER")
DB_PASSWORD = os.getenv("DB_PASSWORD")

ENTRADA = f"postgresql://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_ENTRADA}"
SAIDA = f'postgresql://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_SAI}'

engine_entrada = create_engine(ENTRADA)
engine_saida = create_engine(SAIDA)


query = "SELECT * FROM Administrador"
dados_originais_df = pd.read_sql(query, engine_entrada)
df_anon_admin = dados_originais_df.copy()


In [12]:
def anonimizar_telefone(valor):
    valor_str = str(valor)
    valor_anonimizado = ''.join('*' if caractere.isdigit() else caractere for caractere in valor_str)
    return valor_anonimizado

df_anon_admin["telefone"] = df_anon_admin["telefone"].map(anonimizar_telefone)




In [13]:
def anonimizar_cpf(valor):
    valor_str = str(valor)
    valor_anonimizado = ''.join('*' if caractere.isdigit() else caractere for caractere in valor_str)
    return valor_anonimizado

df_anon_admin["cpf"] = df_anon_admin["cpf"].map(anonimizar_cpf)


In [14]:
def criptografar_senha(senha):
    
    salt = bcrypt.gensalt()
    hash_senha = bcrypt.hashpw(senha.encode('utf-8'), salt)
    return hash_senha.decode('utf-8')

def verificar_senha(senha_digitada, senha_criptografada):

    return bcrypt.checkpw(senha_digitada.encode('utf-8'), senha_criptografada.encode('utf-8'))

df_anon_admin["senha"] = df_anon_admin["senha"].apply(criptografar_senha)






In [15]:

def anonimizar_email(email):
    email = str(email)
    parte_local, dominio = email.split("@")
  
    if len(parte_local) > 3:
        parte_local_pseudo = parte_local[:0] + "****"
    else:
        parte_local_pseudo = parte_local + "****"
    return parte_local_pseudo + "@" + dominio

df_anon_admin["email"] = df_anon_admin["email"].apply(anonimizar_email)


In [16]:
df_anon_admin["nome_completo"] = list(map(
    lambda i: f"Administrador {i+1}", 
    range(len(df_anon_admin))
))




In [17]:
display(df_anon_admin)

,id,telefone,cpf,nome_completo,email,senha
0,1,***********,***********,Administrador 1,****@eitruck.com,$2b$12$hZp/gHKhBhpAIq/6.94Hie3gMea9r2esuN13DyM...
1,2,***********,***********,Administrador 2,****@eitruck.com,$2b$12$lt/VIbLNnoitPXiTD/cuoedjefChhzT8e3RjCZk...
2,3,***********,***********,Administrador 3,****@eitruck.com,$2b$12$OU7/Gzb4QnE5lWRJ/3y.ne6Iz1sDTE9I7s6iywz...
3,4,***********,***********,Administrador 4,****@eitruck.com,$2b$12$QigBXCtF4POWf8pSaUAT5eAXy.LTYYBo3BLOIZk...
4,5,***********,***********,Administrador 5,****@eitruck.com,$2b$12$..1k9X51EKdGqA.ovbFoz.R5U/Yu9E.pTolVuF7...
5,6,***********,***********,Administrador 6,****@eitruck.com,$2b$12$AUjWvyYjVn/oZ6Q4h5FuT.0F6r0n8TU4fxQArZ9...
6,7,***********,***********,Administrador 7,****@eitruck.com,$2b$12$yPFwLcf9DHFoLP1.UrBgrejYhns38dronB.hdLF...
7,8,***********,***********,Administrador 8,****@eitruck.com,$2b$12$GPbOErrHrFXaPJHckXXfAee120wPfj3yxQ/JXgl...
8,9,***********,***********,Administrador 9,****@eitruck.com,$2b$12$kB/YtXyonyXRNG.pQfZ6vuLOG96gID7inu16QKs...
9,10,***********,***********,Administrador 10,****@eitruck.com,$2b$12$ADhqqKcCiWT2G7eGsdPCoenlOBOsd/scgCvo6j2...


In [17]:
dados_originais_df.to_csv('original_administrador.csv')
df_anon_admin.to_csv('pseudo_administrador.csv')

In [18]:
df_anon_admin.to_sql(name='administrador', con=engine_saida, if_exists='replace', index=False)

13